In [157]:
from odlpet import Compression, Scanner
import read_mCT_data #in odlpet > util

In [158]:
import sys
print(sys.version)

3.6.7 | packaged by conda-forge | (default, Nov  6 2019, 16:19:42) 
[GCC 7.3.0]


In [175]:
class my_mct(Scanner):
    det_nx_mm = 4
    det_ny_mm = 2*10*0.2027
    num_rings = 4*13+3 # 1  or (4*13  + 3) # in crystals
    #print(ring_size)
    det_radius = 427.6
    average_depth_of_inter = 9.6
    ring_spacing = det_ny_mm
    voxel_size_xy = 4.07283 #one side, mm
    axial_crystals_per_block = num_rings
    trans_crystals_per_block = 13+1
    axial_blocks_per_bucket = 1
    trans_blocks_per_bucket = 4
    ring_size = 12*trans_blocks_per_bucket*trans_crystals_per_block
    num_dets_per_ring = ring_size

In [176]:
scanner = my_mct()

In [177]:
headers = ["CylinderPET-LM-00-sino-0.s.hdr", "CylinderPET-LM-00.hdr", "narc.hdr", "arc.s.hdr"]

In [178]:
verbose = False
if verbose:
    for hdr in headers: 
        with open(hdr, 'r') as file: print(file.read())

In [179]:
import importlib
importlib.reload(read_mCT_data)

<module 'read_mCT_data' from '/home/mamo/Hugo/read_mCT_data.py'>

In [180]:
def compare(x, y, attributes):
    errors = 0
    for name in attributes:
        if getattr(x, name) != getattr(y, name):
            errors = errors + 1
    return errors == 0

In [181]:
manual_compression = Compression(scanner)
names = ['span_num', 'max_num_segments', 'num_of_views', 'num_non_arccor_bins', 'data_arc_corrected'] 
values = [11, 49, 168, 400, False]
for pair in zip(names, values):
    setattr(manual_compression, *pair)

for k, c in enumerate(automated_compressions):
    is_equal = compare(c, manual_compression, names)
    if is_equal:
        print("automated compression creation successful")
    else:
        print("error, attribute mismatch")
    if k == 2: 
        manual_compression.data_arc_corrected = True

automated compression creation successful
automated compression creation successful
automated compression creation successful
automated compression creation successful


In [182]:
proj = automated_compressions[0].get_projector()
print('domain\n{} \n\nrange\n{}'.format(proj.domain, proj.range)) # for some reason, the image domain is smaller in x, y than Siemens (175 < 200)

domain
uniform_discr([   0.    , -354.3362, -354.3362], [ 220.943 ,  358.4091,  358.4091], (109, 175, 175), dtype='float32') 

range
uniform_discr([   0. ,    0. , -427.6], [ 621.    ,    3.1416,  427.6   ], (621, 168, 400), dtype='float32')
